# Evolutionary Model

A seguir serão discutidos alguns pontos-chave da implementação do modelo, assim como a teoria por trás das decisões de concepções adotadas no projeto. Para a apresentação e discussão dos resultados, a parte executável do código já foi processada e sua saída será utilizada para as conclusões. O motivo disso é o custo computacional inerente do modelo evolutivo, que torna inviável que se aguarde a execução a fim de apreciar os resultados.

# O modelo em si

A concepção do modelo evolutivo dá-se pela concepção de que: sob um determinado ambiente, podemos fazer uma seleção de indivíduos sob um certo viés, o que resulta na disseminação de determinadas características à futuras gerações destes indivíduos.

Para tanto, o ambiente deve seguir determinados critérios e a seleção deve refletir o ambiente:
* Primeiramente, é necessária uma população de indivíduos que irão interagir com o ambiente;
* A estes indivíduos, será utilizada uma função, tradicionalmente chamada de _fitness_, que, a partir de determinados resultados obtidos da interação dos indivíduos com o ambiente, atribuirá uma pontuação ao indivíduos, o que nos permite classificar os indivíduos sob nosso critério de seleção;
* Feita a classificação, propomos que uma nova população será gerada a partir da combinação das características dos indivíduos. Tradicionalmente gera-se uma população nova com a mesma quantidade de indivíduos que a anterior, denominando-se assim como uma nova geração.
Esta nova geração será composta de indivíduos que refletirão características de seus antecessores, sendo então necessária uma concepção de hereditariedade.
* Para que uma nova geração reflita seus antecessores, fazemos tantas seleções de indivíduos antepassados quanto forem necessárias e dessas seleções fazemos a combinação das características, e dessa combinação resulta um novo indivíduo. Em geral, combina-se dois indivíduos antepassados para gerar um novo indivíduo. A seleção de antepassados pode ser realizada de várias formas, a nível de exemplo: neste projeto selecionamos os 5 indivíduo mais bem adaptados e selecionamos 2 ao acaso, sendo que a chance de um indivíduo ser selecionado é proporcional à pontuação de sua função fitness. Repetindo essa seleção de pares quantas vezes forem necessárias, formamos uma nova geração de indivíduos.
* As características dos indivíduos a serem combinadas em geral são valores numérios cuja finalidade varia de implementação para implementação. No nosso modelo alguns valores simbolizam distâncias a serem consideradas, enquanto que outros valores servem como coeficientes para ações a serem tomadas em diversas situações possíveis. A esse conjunto de valores damos o nome de DNA, ou código genético, enquanto que os valores em si são denomindados alelos. A combinação dessas características também é bastante flexível. No nosso código, para cada alelo temos 50% de chance de herdar a característica de algum dos indivíduos selecionados no par.
* Prevendo que características menos eficazes podem disseminar-se contundentemente pelas gerações futuras, incluímos, por fim, uma chance de mutação em algum indivíduo, sendo que essa mutação consiste na alteração de algum alelo para algum valor aleatório que respeite os limites do alelo em questão. No nosso código cada novo indivíduo tem 15% de chance de passar pelo processo de mutação implementado por nós, que, para cada alelo presente no código genético do novo indivíduo, possui 10% de chance de trocar seu valor.

# Implementação do Indivíduo

O código dos indivíduos está escrito no arquivo Agents.py, onde lá encontra-se a classe DumbAgent, que é o agente que possui o comportamento implementado por nós.

No código temos métodos de apoio assim como métodos evocados pelo próprio motor da ferramenta fornecida a nós.

O seguinte trecho de código é executado a cada iteração do jogo. Nele constitui-se o comportamento do agente.
Primeiramente, utilizamos os métodos de apoio para calcular algumas informações como: distância ao fantasma mais próximo, distância a comida mais próxima, assim como suas direções relativas. Uma vez que exista um fantasma próximo o suficiente do pacman, o mesmo dará preferência por evitar os fantasmas, enquanto que caso não haja um fantasma suficientemente próximo, o pacman irá dar preferência a comidas.
No código, quebramos todas as possíveis combinações de entornos que o pacman encontraria a sua volta, e, para estes, o pacman possui diferentes preferências para direções a seguir. Estas preferências são passadas por parâmetro ao pacman, constituindo seu código genético.

    def getAction(self, state):
            "The agent receives a GameState (defined in pacman.py)."
            self.moves += 1
            rw = ReadWrite()
            posG = np.array(state.getGhostPositions())
            posP = np.array(state.getPacmanPosition())
            posCaps = np.array(state.getCapsules())
            posFood = state.getFood()
            
            if(self.numCaps == None):
                self.numCaps = len(posCaps)
            elif(self.numCaps != len(posCaps)):
                self.numCaps = len(posCaps)
                self.Attack = 0
            elif(self.Attack >= rw.pmt[1]):
                self.Attack = None

            if(self.Attack != None):
                self.Attack += 1

            minDistGhost = 99999
            indG = None
            for i in range(len(posG)):
                if(np.linalg.norm(posG[i]-posP) < minDistGhost):
                    minDistGhost = np.linalg.norm(posG[i]-posP)
                    indG =  i

            if(posG[indG][0] > posP[0]):
                if(posG[indG][1] > posP[1]):
                    ghostRelativePosition = "upright"
                elif(posG[indG][1] == posP[1]):
                    ghostRelativePosition = "right"
                else:
                    ghostRelativePosition = "downright"

            elif(posG[indG][0] == posP[0]):
                if(posG[indG][1] > posP[1]):
                    ghostRelativePosition = "up"
                else:
                    ghostRelativePosition = "down"
            else:
                if(posG[indG][1] > posP[1]):
                    ghostRelativePosition = "upleft"
                elif(posG[indG][1] == posP[1]):
                    ghostRelativePosition = "left"
                else:
                    ghostRelativePosition = "downleft"

            minDistFood = 99999
            minFood = None
            for x in range(posFood.width):
                for y in range(posFood.height):
                    if(posFood[x][y]):
                        food = np.array([x,y])
                        distFood = np.linalg.norm(food-posP)
                        if(distFood<minDistFood):
                            minDistFood = distFood
                            minFood = food
            
            if(minFood[0] > posP[0]):
                if(minFood[1] > posP[1]):
                    foodRelativePosition = "upright"
                elif(minFood[1] == posP[1]):
                    foodRelativePosition = "right"
                else:
                    foodRelativePosition = "downright"

            elif(minFood[0] == posP[0]):
                if(minFood[1] > posP[1]):
                    foodRelativePosition = "up"
                else:
                    foodRelativePosition = "down"
            else:
                if(minFood[1] > posP[1]):
                    foodRelativePosition = "upleft"
                elif(minFood[1] == posP[1]):
                    foodRelativePosition = "left"
                else:
                    foodRelativePosition = "downleft"

            if(len(state.getLegalPacmanActions()) == 2 ):
                path = "deadend"
            else:
                path = None

            if(path == "deadend"):
                if (Directions.SOUTH in state.getLegalPacmanActions()):
                    return Directions.SOUTH
                elif (Directions.NORTH in state.getLegalPacmanActions()):
                    return Directions.NORTH
                elif (Directions.WEST in state.getLegalPacmanActions()):
                    return Directions.WEST
                else:
                    return Directions.EAST

            elif(minDistGhost < rw.pmt[0] and self.Attack == None):
                if(ghostRelativePosition == "right"):
                    return self.directions(state,self.PossibleDirections[rw.pmt[2]])

                elif(ghostRelativePosition == "left"):
                    return self.directions(state,self.PossibleDirections[rw.pmt[3]])

                elif(ghostRelativePosition == "up"):
                    return self.directions(state,self.PossibleDirections[rw.pmt[4]])

                elif(ghostRelativePosition == "down"):
                    return self.directions(state,self.PossibleDirections[rw.pmt[5]])

                elif(ghostRelativePosition == "downright"):
                    return self.directions(state,self.PossibleDirections[rw.pmt[6]])

                elif(ghostRelativePosition == "downleft"):
                    return self.directions(state,self.PossibleDirections[rw.pmt[7]])
                
                elif(ghostRelativePosition == "upleft"):
                    return self.directions(state,self.PossibleDirections[rw.pmt[8]])
                
                elif(ghostRelativePosition == "upright"):
                    return self.directions(state,self.PossibleDirections[rw.pmt[9]])
            else:
                if(foodRelativePosition == "right"):
                    return self.directionsProb(state=state,p1=rw.pmt[10],p2=rw.pmt[11],p3=rw.pmt[12],p4=rw.pmt[13])

                elif(foodRelativePosition == "left"):
                    return self.directionsProb(state=state,p1=rw.pmt[14],p2=rw.pmt[15],p3=rw.pmt[16],p4=rw.pmt[17])

                elif(foodRelativePosition == "up"):
                    return self.directionsProb(state=state,p1=rw.pmt[18],p2=rw.pmt[19],p3=rw.pmt[20],p4=rw.pmt[21])

                elif(foodRelativePosition == "down"):
                    return self.directionsProb(state=state,p1=rw.pmt[22],p2=rw.pmt[23],p3=rw.pmt[24],p4=rw.pmt[25])

                elif(foodRelativePosition == "downright"):
                    return self.directionsProb(state=state,p1=rw.pmt[26],p2=rw.pmt[27],p3=rw.pmt[28],p4=rw.pmt[29])

                elif(foodRelativePosition == "downleft"):
                    return self.directionsProb(state=state,p1=rw.pmt[30],p2=rw.pmt[31],p3=rw.pmt[32],p4=rw.pmt[33])
                
                elif(foodRelativePosition == "upleft"):
                    return self.directionsProb(state=state,p1=rw.pmt[34],p2=rw.pmt[35],p3=rw.pmt[36],p4=rw.pmt[37])
                
                elif(foodRelativePosition == "upright"):
                    return self.directionsProb(state=state,p1=rw.pmt[38],p2=rw.pmt[39],p3=rw.pmt[40],p4=rw.pmt[41])


Outro método importante está no arquivo ReadWrite.py, mais especificamente o construtor da classe. No construtor lemos os parâmetros que compõem o código genético do indivíduo. Perceba a manipulação de diversos arquivos .txt que foram utilizado para a comunicação entre diferentes módulos da ferramenta.
        def __init__(self):

            with open('nome.txt', 'r') as f:
                lines = f.readlines()
                self.nome = lines[0]

            with open('agentNum_' + self.nome + '.txt') as f:
                lines = f.readlines()
                self.agentNum = int(lines[0])

            with open('parameters_' + self.nome + '.txt') as f:
                lines = f.readlines()
                
                self.pmt = lines[self.agentNum].split()

                for i in range(len(self.pmt)):
                    if(1<=i<=9):
                        self.pmt[i] = int(self.pmt[i])
                    else:
                        self.pmt[i] = float(self.pmt[i])

Com isso chegamos ao ultimo método aqui listado, pertencente à classe DumbAgent. Temos aqui o metodo final(), que é evocado pela ferramenta no final de cada jogo, e com isso escrevendo resultados relativos a cada indivíduo da população.

        def final(self, state):
            posCaps = np.array(state.getCapsules())

            with open('moves.txt', 'w') as f:
                f.write(str(self.moves) + '\n')

            print ("Acabou",posCaps)
    


# Implementação do modelo

Para a implementação do modelo e seu funcionamento, utilizamos diversos métodos de apoio localizados mais adiante neste notebook.

A começar pelas dependências de importações: 

        from numpy.random import choice
        import numpy as np
        import random
        import matplotlib.pyplot as plt

A seguir temos um dos métodos mais importantes, o método crossOver(). Nele fazemos a combinação de dois indivíduos (p1 e p2), gerando por fim um novo indivíduo (child).

        def crossOver(p1,p2):
                dadOrmom = [False, True]
                child = []

                cross = choice(dadOrmom, p=[0.5,0.5])

                if(cross):
                        child.append(p1[0])
                else:
                        child.append(p2[0])
                
                for i in range(1,10):
                        cross = choice(dadOrmom, p=[0.5,0.5])
                        if(cross):
                        child.append(p1[i])
                        else:
                        child.append(p2[i])

                for i in range(10,18):
                        cross = choice(dadOrmom, p=[0.5,0.5])

                        if(cross):
                
                                index = (i-9)*4+9
                                child.append(p1[index-3])
                                child.append(p1[index-2])
                                child.append(p1[index-1])
                                child.append(p1[index-0])
                        else:
                        
                                index = (i-9)*4+9
                                child.append(p2[index-3])
                                child.append(p2[index-2])
                                child.append(p2[index-1])
                                child.append(p2[index-0])
                return child

Aqui está um de nosso métodos de apoio, que gera 4 numeros aleatórios cuja soma resulta em 1

                def random4():
                        p1 = np.random.uniform(low=0.0, high=1)
                        p2 = np.random.uniform(low=0.0, high=(1-p1))
                        p3 = np.random.uniform(low=0.0, high=(1-p1-p2))
                        p4 = 1-(p1+p2+p3)
                        while(p1+p2+p3+p4!=1):
                                p1 = np.random.uniform(low=0.0, high=1)
                                p2 = np.random.uniform(low=0.0, high=(1-p1))
                                p3 = np.random.uniform(low=0.0, high=(1-p1-p2))
                                p4 = 1-(p1+p2+p3)
                        return p1,p2,p3,p4

Em sequência temos o método responsável pela mutação do novo indivíduo (child). Nele, cada alelo possui 10% de chance de ser alterado respeitando os valores limites que ele pode assumir.

                def mutationChild(child):
                        mutation = [False, True]

                        ifMut = choice(mutation, p=[0.9,0.1])
                        if(ifMut):
                                child[0] = np.random.uniform(low=0.0, high=5.0)
                        
                        for i in range(1,10):
                                ifMut = choice(mutation, p=[0.9,0.1)
                                if(ifMut):
                                if(i == 1):
                                        child[i] = random.randint(0, 30)
                                else:
                                        child[i] = random.randint(0, 23)
                        
                        for i in range(10,18):
                                ifMut = choice(mutation, p=[0.9,0.1)
                                if(ifMut):
                                
                                index = (i-9)*4+9
                                p1,p2,p3,p4 = random4()
                                child[index-3] = p1
                                child[index-2] = p2
                                child[index-1] = p3
                                child[index-0] = p4
                        return child      

Temos então mais um método de apoio que nos auxilia na conversão de listas em strings.

                def writeString(child):
                        string = ""
                        for i in range(len(child)):
                                string += str(child[i])+" "
                        string+="\n"
                        return string

Por fim temos o método geraPopIni, que gera combinações aleatórias de códigos genéticos para serem utilizados como geração inicial do experimento.

                def geraPopIni(nome, pop):   
                        import numpy as np
                        import random
                        

                        string = "" + str(np.random.uniform(low=0.0, high=5.0))

                        string += " " + str(random.randint(0, 30))

                        for i in range(8):
                                string += " " + str(random.randint(0, 23))

                        for i in range(8):
                                p1,p2,p3,p4 = random4()
                                string += " " + str(p1) + " " + str(p2) + " " + str(p3) + " " + str(p4)

                        with open('parameters_' + nome + '.txt','w') as f:
                                f.write(string)



                        for i in range(pop - 1):

                                string = "\n" + str(np.random.uniform(low=0.0, high=5.0))

                                string += " " + str(random.randint(0, 30))

                                for i in range(8):
                                string += " " + str(random.randint(0, 23))

                                for i in range(8):
                                p1,p2,p3,p4 = random4()
                                string += " " + str(p1) + " " + str(p2) + " " + str(p3) + " " + str(p4)
                                        
                                with open('parameters_' + nome + '.txt','a') as f:
                                f.write(string)




# Execução do experimento

Arquitetado todo o aparato necessário para os experimentos, podemos então utilizá-lo num único método que será responsável por reproduzir o que foi mencionado na seção do modelo genético:  
* Inicializamos as variáveis necessárias, assim como os arquivos de saída necessários. É gerada também a população inicial do experimento utilizando o método geraPopIni();
* 

In [29]:
def treina(layout, nGames, minVic, pop):
    from numpy.random import choice
    import numpy as np
    import time

    start_time = time.time()

    population = pop
    count = 6
    NotStop = True
    hasBest = False

    bCount = 0

    nome = layout
    geraPopIni(nome, pop)

    with open('nome.txt', 'w') as f:
        f.write(nome)

    generation = 0
    with open('generation_' + nome + '.txt','w') as f:
        f.write("")

    with open('fitness_' + nome + '.txt','w') as f:
        f.write("")

    with open('Best_' + nome + '.txt','w') as f:
        f.write("")

    while(NotStop):

        # Run games

        with open('results_' + nome + '.txt','w') as f:
                f.write("")
                f.close()
        for i in range(population):

            with open('agentNum_' + nome + '.txt','w') as f:
                f.write(str(i))

            saida = !python pacman.py --layout {layout} --pacman DumbAgent --numGames {nGames} --quietTextGraphics
            
        allPmt = []
        allRes = []
        allRes2 = []
        inds = []

        # Read parameters 
        for k in range (population):
            with open('parameters_' + nome + '.txt') as f:
                lines = f.readlines()

                pmt = lines[k].split()

            for j in range(len(pmt)):
                if(1<=j<=9):
                    pmt[j] = int(pmt[j])
                else:
                    pmt[j] = float(pmt[j])
            
            allPmt.append(pmt)

        # Read results
        for k in range (population):
            with open('results_' + nome + '.txt') as f:
                lines = f.readlines()

                res = lines[k].split()

            for j in range(len(res)):
                res[j] = float(res[j])
            
            allRes.append(res)


        # Select the best 5

        fitList = np.array([])

        for k in range (population):
            fit = allRes[k][1]+((229-allRes[k][3])/229)*2 + ((4-allRes[k][4])/4) + (min(allRes[k][5],1))
            fitList = np.append(fitList,fit)

        inds = np.argpartition(fitList, -5)[-5:]
        indsWorsts = np.argpartition(fitList, 5)[:5]

        print("Gen: ",generation)
        print("Best 5:")
        for i in inds:
            print(i,fitList[i],allRes[i])
        print("Worst 5:")
        for i in indsWorsts:   
            print(i,fitList[i],allRes[i])
        print()

        avgFit = sum(fitList)/len(fitList)
        avgBestFit = 0
        avgWorstFit = 0

        for i in inds:
            avgBestFit += fitList[i]
        avgBestFit = avgBestFit/5
        for i in indsWorsts:
            avgWorstFit+= fitList[i]
        avgWorstFit = avgWorstFit/5

        with open('fitness_' + nome + '.txt','a') as f:
            s = str(avgBestFit) + " " + str(avgFit) + " " + str(avgWorstFit) + "\n"
            f.write(s)

        for i in inds:
            if(allRes[i][1]>=minVic):

                if hasBest:                  

                    if fitList[i] > best[0] and allRes[i][0] > best[1] and allRes[i][1] > best[2]:
                        best = [fitList[i], allRes[i][0], allRes[i][1]]
                        with open('Best_' + nome + '.txt','w') as f:
                            f.write(writeString(allPmt[i]))
                            f.write(writeString([fitList[i])])
                            f.write(writeString(allRes[i]))
                            spentTime = (time.time() - start_time)
                            f.write("Execution Time: "+str(spentTime) + '\n')
                        bCount += 1
                        count = 6                  

                else:
                    hasBest = True

                    best = [fitList[i], allRes[i][0], allRes[i][1]]
                    bCount += 1
                    with open('Best_' + nome + '.txt','w') as f:
                        f.write(writeString(allPmt[i]))
                        f.write(writeString([fitList[i])])
                        f.write(writeString(allRes[i]))
                        spentTime = (time.time() - start_time)
                        f.write("Execution Time: "+str(spentTime) + '\n')
        
        if (hasBest):
            count -= 1
            if count == 0 or bCount == 10:
                NotStop = False 

        if(NotStop):

            with open('parameters_' + nome + '.txt','w') as f:
                f.write("")


            # CrossOver

            childs = []

            for k in range(int(population)):               
                d = choice(inds, p=fitList[inds]/fitList[inds].sum())
                m = choice(inds, p=fitList[inds]/fitList[inds].sum())
                while(d==m):
                    m = choice(inds, p=fitList[inds]/fitList[inds].sum())


                child = crossOver(allPmt[d],allPmt[m])

                # Mutation
                if choice([True, False], p=[0.15, 0.85]):
                    child = mutationChild(child)


                childs.append(child)

            with open('parameters_' + nome + '.txt','a') as f:
                for k in range(population):
                    f.write(writeString(childs[k]))

            

            with open('generation_' + nome + '.txt','a') as f:
                f.write(str(generation)+"\n")
            generation+=1





In [30]:
!python pacman.py --help

Usage: 
    USAGE:      python pacman.py <options>
    EXAMPLES:   (1) python pacman.py
                    - starts an interactive game
                (2) python pacman.py --layout smallClassic --zoom 2
                OR  python pacman.py -l smallClassic -z 2
                    - starts an interactive game on a smaller board, zoomed in
    

Options:
  -h, --help            show this help message and exit
  -n GAMES, --numGames=GAMES
                        the number of GAMES to play [Default: 1]
  -l LAYOUT_FILE, --layout=LAYOUT_FILE
                        the LAYOUT_FILE from which to load the map layout
                        [Default: mediumClassic]
  -p TYPE, --pacman=TYPE
                        the agent TYPE in the pacmanAgents module to use
                        [Default: KeyboardAgent]
  -t, --textGraphics    Display output as text only
  -q, --quietTextGraphics
                        Generate minimal output and no graphics
  -g TYPE, --ghosts=TYPE
                 

In [31]:
# !pacman.py --layout smallClassic -p DumbAgent --numGames 10

In [32]:
nGames = {"smallClassic" : 9, "mediumClassic" : 6, "originalClassic" : 3}
minVic = {"smallClassic" : 6, "mediumClassic" : 4, "originalClassic" : 2}
pops =   {"smallClassic" : 200, "mediumClassic" : 100, "originalClassic" : 50}

for layout in ["smallClassic", "mediumClassic", "originalClassic"]:
    treina(layout, nGames[layout], minVic[layout], pops[layout])

Gen:  0
Best 5:
12 2.4149965980799757 [-348.0, 0.0, 0.0, 37.1111111111, 1.66666666667, 0.155777772268, 71.3333333333]
168 2.4458563761587118 [-369.555555556, 0.0, 0.0, 35.3333333333, 1.55555555556, 0.143333329095, 66.2222222222]
199 2.534316357219131 [-306.111111111, 0.0, 0.0, 32.8888888889, 1.77777777778, 0.266000005934, 116.111111111]
130 2.679014555404548 [-342.888888889, 0.0, 0.0, 26.3333333333, 1.66666666667, 0.325666666031, 151.777777778]
149 3.3047671033472983 [-250.111111111, 1.0, 0.111111111111, 36.6666666667, 1.88888888889, 0.0972222222222, 44.5555555556]
Worst 5:
87 2.0845327491842616 [-498.0, 0.0, 0.0, 52.8888888889, 2.0, 0.0464444425371, 19.1111111111]
94 2.100969425744752 [-522.555555556, 0.0, 0.0, 54.1111111111, 2.0, 0.0735555489858, 31.4444444444]
2 2.113926752396843 [-445.888888889, 0.0, 0.0, 48.1111111111, 2.0, 0.034111128913, 14.7777777778]
97 2.1184672400072557 [-458.666666667, 0.0, 0.0, 48.8888888889, 2.0, 0.0454444355435, 19.7777777778]
129 2.118748654676094 [-460

In [ ]:
for layout in ["smallClassic", "mediumClassic", "originalClassic"]:
        Best = []
        Avg = []
        Wrst = []
        with open('fitness_' + layout + '.txt', 'r') as f:
            lines = f.readlines()
            for line in lines:
                line = line.split()
                Best.append(float(line[0]))
                Avg.append(float(line[1]))
                Wrst.append(float(line[2]))
            
            x = np.arange(len(lines))
           
        
        plt.plot(x, Best, label = 'Best') 
        plt.plot(x, Avg, label = 'Average')
        plt.plot(x, Wrst, label = 'Worst')
        plt.legend()
        plt.title("Gráfico de Fitness ao longo das gerações\n* {} *".format(layout))
        plt.xlabel('Geração')
        plt.ylabel('Fitness')
        plt.show()